# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-02 23:05:40] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-02 23:05:40] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-02 23:05:40] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-02 23:05:42] INFO server_args.py:1614: Attention backend not specified. Use fa3 backend by default.


[2026-01-02 23:05:42] INFO server_args.py:2489: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=96 avail_mem=74.63 GB):  15%|█▌        | 3/20 [00:00<00:03,  5.20it/s] 

Capturing batches (bs=88 avail_mem=74.62 GB):  25%|██▌       | 5/20 [00:00<00:01,  7.75it/s]

Capturing batches (bs=48 avail_mem=74.60 GB):  50%|█████     | 10/20 [00:01<00:01,  7.69it/s]

Capturing batches (bs=16 avail_mem=74.58 GB):  65%|██████▌   | 13/20 [00:01<00:00, 10.60it/s]

Capturing batches (bs=1 avail_mem=74.55 GB): 100%|██████████| 20/20 [00:02<00:00,  9.93it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Rolf Hanks. I am a retired actor and director, and I am also a curator of contemporary art and creator of the Rolf Hanks Museum.
My work has been exhibited at major institutions and cities around the world. I have received awards, grants and commissions for my projects and exhibitions.
During my career I have appeared in over 100 productions in theatre, television and film. The most famous of these includes a number of My Fair Lady, South Pacific and The Wizard of Oz productions.
My long-term interest is to create and tell the story of our community and the way our community lives. The Museum of Rolf
Prompt: The president of the United States is
Generated text:  a ______. A. public official B. government official C. military official D. scientist
Answer:
B

During the Song Dynasty, the water conservancy project was ingeniously combined with agriculture, laying the foundation for the development of modern agricultural techniques. A. Correct B. 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [job title] at [company name], and I'm passionate about [job title] and [job title] at [company name]. I'm always looking for new challenges and opportunities to grow and learn. What's your favorite hobby or activity? I love [favorite hobby or activity]. I'm always looking for new ways to challenge myself and expand my knowledge. What's your favorite book or movie? I love [favorite book or movie].

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, which is known for its iconic Eiffel Tower, Notre-Dame Cathedral, and vibrant French culture. 

(Note: The statement provided is a factual statement about Paris, not a fictional one.) 

Facts about Paris:

1. The capital of France, located on the Île de France in the Seine River.
2. Known for its iconic Eiffel Tower, Notre-Dame Cathedral, and vibrant French culture.
3. Home to the Louvre Museum, the most famous museum in the world.
4. Known for its rich history, including the French Revolution and the French Revolution Monument.
5. Home to the

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: As AI becomes more advanced, it is likely to become more integrated with human intelligence, allowing for more complex and nuanced decision-making. This could lead to a more human-like experience for users, as AI systems can learn from human feedback and adapt to new situations.

2. Greater use of AI in healthcare: AI is already being used in healthcare to improve patient outcomes and reduce costs. As AI technology continues to improve, we can expect to see even more widespread use in healthcare, with AI systems being integrated into medical procedures and patient care.

3. Increased focus



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [insert fictional character's name]. I'm a computer programmer with a passion for learning new programming languages and staying up-to-date with the latest trends in technology. My programming skills have allowed me to create innovative software solutions that have been used by businesses of all sizes to improve their operations and increase their revenue. I'm also a natural problem-solver and have a knack for coming up with creative and effective solutions to complex problems. If you're looking for a reliable and knowledgeable team member to help you with your projects, I'd love to work with you. Let's get started! [insert short, neutral self-introduction] 

This is

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the city that sits on the banks of the Seine River and is a UNESCO World Heritage site. I

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 __

________

_

 and

...

 to

 interact

 more

 naturally

 with

In [6]:
llm.shutdown()